# load data

In [1]:
import numpy as np

In [2]:
class DataLoader:

    def __init__(self, path):
        """
        :param path: File path of the data
        """
        data = np.loadtxt(path, delimiter=',')
        np.random.seed(1)
        np.random.shuffle(data)
        self.X = data[:, :21]
        self.Y = data[:, 21:]

    def get_all_data(self, normalize=False):
        """
        Returns the entire data set.
        :param normalize: Whether or not the data will be normalized
        :return: Data set, output values
        """
        if normalize:
            return DataLoader._normalize(self.X), self.Y
        return self.X, self.Y

    def get_data_split(self, split, normalize=False):
        """
        Returns the data according to the specified splits.
        The 'split' parameter is a vector with either two or three components that in both cases have to sum to 1.
        If the 'split' vector contains two components, the first component specifies the portion of the data set
        that will be used for the training set and the second component specifies the portion of the data set that
        will be used for the test set.
        If the 'split' vector contains three components, the first and last components will be interpreted as described
        above and the middle component will be used to specify the portion of the data set that will be used for the
        cross validation set.
        :param split: A vector containing either 2 or 3 values.
        :param normalize: Whether or not the data will be normalized
        :return: Training set, output values for training set, cross validation set, output values for cross validation
                    set, test set, output values for cross validation set
        """
        assert np.sum(split) == 1
        X = self.X
        if normalize:
            X = DataLoader._normalize(self.X)
        if len(split) == 2:
            idx_train = int(np.round(self.X.shape[0] * split[0]))
            X_train = X[:idx_train, :]
            Y_train = self.Y[:idx_train, :]
            X_test = X[idx_train:, :]
            Y_test = self.Y[idx_train:, :]
            return X_train, Y_train, X_test, Y_test
        if len(split) == 3:
            idx_train = int(np.round(self.X.shape[0] * split[0]))
            idx_cv = idx_train + int(np.round(self.X.shape[0] * split[1]))
            idx_test = idx_cv + int(np.round(self.X.shape[0] * split[2]))
            X_train = X[:idx_train, :]
            Y_train = self.Y[:idx_train, :]
            X_cv = X[idx_train:idx_cv, :]
            Y_cv = self.Y[idx_train:idx_cv, :]
            X_test = X[idx_cv:idx_test, :]
            Y_test = self.Y[idx_cv:idx_test, :]
            return X_train, Y_train, X_cv, Y_cv, X_test, Y_test

    @staticmethod
    def _normalize(X):
        """
        Normalizes the data.
        :param X: Data
        :return: Normalized data
        """
        return (X - np.mean(X, axis=0)) / np.std(X, axis=0)


# Nearest neighbour 

In [3]:
class NearestNeighbors:

    def __init__(self, X_train, Y_train):
        """
        :param X_train: training set
        :param Y_train: outputs of the training set
        """
        self.X_train = X_train
        self.Y_train = Y_train

    def predict(self, x, k):
        """
        Predicts the output value of the provided example.
        :param x: Example input vector
        :param k: The number of neighbors used for computing the prediction
        :return: Prediction made based on the k neighbors
        """
        dist = np.linalg.norm(self.X_train - x, axis=1)
        idx = np.argpartition(dist, k)[:k]
        weights = 1 / dist[idx]**2
        return np.sum(np.squeeze(self.Y_train[idx]) * np.squeeze(weights)) / np.sum(weights)
        #return np.sum(np.squeeze(self.Y_train[idx])) / k

    def test(self, X_test, Y_test, k):
        """
        Tests the performance of the algorithm by means of the provided test set.
        :param X_test: Test set
        :param Y_test: Output values for the test set
        :param k: The number of neighbors used for computing the prediction
        :return: RMSE of the test set, predictions made on the test set
        """
        X_pred = np.zeros(Y_test.shape)
        for i in range(X_pred.shape[0]):
            X_pred[i][0] = self.predict(X_test[i], k)
        rmse = np.sqrt(np.mean(np.square(X_pred - Y_test)))
        return rmse, X_pred

    def get_best_k(self, X, Y, limit):
        """
        Tries out values for k in the range from 1 to 'limit', returns the k that yields the lowest RMSE.
        :param X: Cross validation set
        :param Y: Output values for the cross validation set
        :param limit: Range limit for k
        :return: Optimal k for the given range
        """
        best_k = -1
        best_rmse = np.inf
        for k in range(1, limit + 1):
            rmse, _ = self.test(X, Y, k)
            if rmse < best_rmse:
                best_rmse = rmse
                best_k = k
        return best_k

# Linear regression

In [4]:
class LinearRegression:

    def __init__(self, X_train, Y_train):
        """
        :param X_train: training set
        :param Y_train: outputs of the training set
        """
        self.X_train = X_train
        self.Y_train = Y_train
        n = self.X_train.shape[1]
        self.w = np.random.normal(0, 0.5, size=(n, 1))
        self.b = np.random.normal(0, 0.5)

    def train(self, iterations, learning_rate, print_iterations=False):
        """
        Optimizes the objective function with the gradient descent algorithm
        in order to find well suited parameters w and b.
        :param iterations: The number of gradient descent iterations
        :param learning_rate: Step size of the gradient descent algorithm
        :param print_iterations: Whether or not the current iteration and its training loss shall be printed to console
        """
        cost_prev = np.inf
        current_learning_rate = learning_rate
        for _ in range(iterations):
            prediction = np.dot(self.X_train, self.w) + self.b
            cost = self._cost(prediction)
            if print_iterations:
                print('Iteration ' + str(_) + ', cost: ' + str(cost))
            self._update_weights(learning_rate, prediction)

            if cost >= cost_prev:
                current_learning_rate = current_learning_rate * 0.5
                continue

            if np.fabs(cost - cost_prev) < 1e-12:
                print('Stopping early')
                break

            cost_prev = cost

    def test(self, X_test, Y_test):
        """
        Test the current parameters on the provided test set.
        :param X_test: Test set
        :param Y_test: Output values for the test set
        :return: RMSE for the test set, predictions on the test set
        """
        prediction = np.dot(X_test, self.w) + self.b
        rmse = np.sqrt(np.mean(np.square(prediction - Y_test)))
        return rmse, prediction

    def _update_weights(self, learning_rate, prediction):
        """
        Implements the parameter update in the direction of the negative
        gradient of the objective function.
        :param learning_rate: The step size
        :param prediction: The predictions from the current iteration
        :return:
        """
        grad_w, grad_b = self._compute_gradient(prediction)
        self.w = self.w - learning_rate * grad_w
        self.b = self.b - learning_rate * grad_b

    def _compute_gradient(self, prediction):
        """
        Computes the gradient for the parameters w and b.
        :param prediction: The predictions from the current iteration
        :return: gradient of w, gradient of b
        """
        m = prediction.shape[0]
        grad_w = np.dot(self.X_train.T, (prediction - self.Y_train)) / m
        grad_b = np.sum((prediction - self.Y_train)) / m
        return grad_w, grad_b

    def _cost(self, prediction):
        """
        Computes the cost for the current set of parameters.
        :param prediction: The predictions from the current iteration
        :return: Cost value
        """
        return np.sum((prediction - self.Y_train)**2) / (2 * self.Y_train.shape[0])

    def reset_parameters(self):
        """
        Resets the parameters of the model.
        """
        n = self.X_train.shape[1]
        self.w = np.random.normal(0, 0.5, size=(n, 1))
        self.b = np.random.normal(0, 0.5)

    def normal_equation(self, X_test, Y_test):
        """
        Computes the exact solution for the linear regression problem via the normal equation.
        :param X_test: Test set
        :param Y_test: Output values of the test set
        :return: RMSE of the test set, predictions for the test set
        """
        X = np.copy(self.X_train)
        X = np.append(np.ones((X.shape[0], 1)), X, axis=1)
        w = np.dot(np.dot(np.dot(X.T, X), X.T), self.Y_train)
        X_ = np.copy(X_test)
        X_ = np.append(np.ones((X_.shape[0],1)), X_, axis=1)
        print(w.shape)
        print(X_.shape)
        prediction = np.dot(X_, w)
        rmse = np.sqrt(np.mean(np.square(prediction - Y_test)))
        return rmse, prediction

# Regression forest

In [5]:
class Node:

    def __init__(self, is_leaf, mean, std, feature, split, left, right):
        """
        :param is_leaf: Whether or not it is a leaf node
        :param mean: The current mean of all the examples contained in the node
        :param std: The current standard deviation of all the examples contained in the node
        :param feature: The feature by which the split is performed
        :param split: The actual threshold value for the split
        :param left: Left child node
        :param right: Right child node
        """
        self.is_leaf = is_leaf
        self.mean = mean
        self.std = std
        self.feature = feature
        self.split = split
        self.left = left
        self.right = right


class DecisionTree:

    def __init__(self, X_train, Y_train, min_nodes):
        """
        :param X_train: Training set
        :param Y_train: Output values for the training set
        :param min_nodes: If a node contains equal or less than 'min_nodes',
                            then the recursion is terminated and a leaf node is created
        """
        self.root = self._create_tree(X_train, Y_train, min_nodes)

    def predict(self, x):
        """
        Predicts the output value of the input vector.
        :param x: Input vector
        :return: Mean of the leaf node, standard deviation of the leaf node
        """
        return self._predict(self.root, x)

    def predict_all(self, X):
        """
        Predicts the output values of an entire set of input vectors.
        :param X: Set of input vectors
        :return: Mean of the leaf nodes, standard deviation of the leaf nodes
        """
        means = np.zeros((X.shape[0], 1))
        stds = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            mean, std = self._predict(self.root, X[i])
            means[i] = mean
            stds[i] = std
        return means, stds

    def _predict(self, node, x):
        """
        Helper function for the predict function.
        :param node: Current node in the tree
        :param x: Input vector for which we predict the output value
        :return: Mean of the leaf node, standard deviation of the leaf node
        """
        if node.is_leaf:
            return node.mean, node.std
        if x[node.feature] <= node.split:
            return self._predict(node.left, x)
        else:
            return self._predict(node.right, x)

    def _create_tree(self, X, Y, min_nodes):
        """
        Recursively creates a decision tree.
        :param X: Current set of training examples
        :param Y: Output values for the training examples
        :param min_nodes: If a node contains equal or less than 'min_nodes', then
                            the recursion is terminated and a leaf node is created
        :return: Eventually returns the root of the decision tree
        """
        if len(X) <= min_nodes:
            return Node(is_leaf=True, mean=np.mean(Y), std=np.std(Y), feature=None, split=None, left=None, right=None)
        feature, best_split, left_indices, right_indices = DecisionTree._best_split_random(X, Y)
        left = self._create_tree(X[left_indices], Y[left_indices], min_nodes)
        right = self._create_tree(X[right_indices], Y[right_indices], min_nodes)
        return Node(is_leaf=False, mean=np.mean(Y), std=np.std(Y), feature=feature, split=best_split, left=left, right=right)

    @staticmethod
    def _best_split_random(X, Y):
        """
        Computes the best split for a given set of training examples.
        Tries out only a random subset (of size num_features / 2 + 1) of the features.
        Tries out only a random subset (of size sqrt(num_values)*2) of all of the existing values for a specific feature.
        :param X: Set of training examples
        :param Y: Output values of training examples
        :return: Feature for split, threshold value for split, indices of left child, indices of right child
        """
        min_sse = np.inf
        feature = None
        best_split = None
        left = None
        right = None
        random_feature_indices_ = np.random.choice(np.arange(X.shape[1]), size=int(X.shape[1] // 2 + 1), replace=False)
        for j in range(len(random_feature_indices_)):
            i = random_feature_indices_[j]
            num_values = X[:, i].shape[0]
            random_values_indices = np.random.choice(np.arange(num_values), size=int(2 * np.sqrt(num_values)), replace=False)
            values = X[random_values_indices, i]
            for split in values:
                left_indices = X[:, i:(i+1)] <= split
                left_indices, _ = np.nonzero(left_indices)
                right_indices = X[:, i:(i+1)] > split
                right_indices, _ = np.nonzero(right_indices)
                sse = DecisionTree._sum_of_squared_errors(Y[left_indices], Y[right_indices])
                if sse < min_sse:
                    feature = i
                    best_split = split
                    left = left_indices
                    right = right_indices
                    min_sse = sse
        return feature, best_split, left, right

    @staticmethod
    def _best_split(X, Y):
        """
        Computes the best split for a given set of training examples.
        Tries out all of the existing values for a specific feature.
        :param X: Set of training examples
        :param Y: Output values of training examples
        :return: Feature for split, threshold value for split, indices of left child, indices of right child
        """
        min_sse = np.inf
        feature = None
        best_split = None
        left = None
        right = None
        for i in range(X.shape[1]):
            values = np.unique(X[:, i])
            for split in values:
                left_indices = X[:, i:(i+1)] <= split
                left_indices, _ = np.nonzero(left_indices)
                right_indices = X[:, i:(i+1)] > split
                right_indices, _ = np.nonzero(right_indices)
                sse = DecisionTree._sum_of_squared_errors(Y[left_indices], Y[right_indices])
                if sse < min_sse:
                    feature = i
                    best_split = split
                    left = left_indices
                    right = right_indices
                    min_sse = sse
        return feature, best_split, left, right

    @staticmethod
    def _sum_of_squared_errors(Y1, Y2):
        """
        Computes the sum of squared errors between two vectors.
        :param Y1: First vector
        :param Y2: Second vector
        :return: Sum of squared errors
        """
        if len(Y1) == 0:
            sse1 = 0
        else:
            sse1 = np.sum((Y1 - np.mean(Y1))**2)
        if len(Y2) == 0:
            sse2 = 0
        else:
            sse2 = np.sum((Y2 - np.mean(Y2)) ** 2)
        return sse1 + sse2


In [6]:
class RandomForest:

    def __init__(self, X_train, Y_train, num_trees, training_set_size, min_nodes):
        """
        :param X_train: Training set
        :param Y_train: Output values for the training set
        :param num_trees: Number of decision trees
        :param training_set_size: Size of the training set that each decision tree is build with
        :param min_nodes: If a node contains equal or less than 'min_nodes',
                            then the recursion is terminated and a leaf node is created
        """
        self.X_train = X_train
        self.Y_train = Y_train
        self.tree_list = []
        self._create_forest(num_trees, training_set_size, min_nodes)

    def predict(self, x):
        """
        Predicts the output value of the input vector using
        the average over all of the prediction from the decision trees.
        :param x: Input vector
        :return: Mean of the leaf nodes, standard deviation of the leaf nodes
        """
        mean_all = 0
        std_all = 0
        for tree in self.tree_list:
            mean, std = tree.predict(x)
            mean_all = mean_all + mean
            std_all = std_all + std
        mean_all = mean_all / len(self.tree_list)
        std_all = std_all / len(self.tree_list)
        return mean_all, std_all

    def predict_all(self, X):
        """
        Predicts the output values of an entire set of input vectors
        using the average over all of the prediction from the decision trees.
        :param X: Set of input vectors
        :return: Mean of the leaf nodes, standard deviation of the leaf nodes
        """
        means = np.zeros((X.shape[0], 1))
        stds = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            mean, std = self.predict(X[i])
            means[i] = mean
            stds[i] = std
        return means, stds

    def _create_forest(self, num_trees, training_set_size, min_nodes):
        """
        Creates a random forest.
        :param num_trees: Number of decision trees
        :param training_set_size: Size of the training set that each decision tree is build with
        :param min_nodes: If a node contains equal or less than 'min_nodes',
                            then the recursion is terminated and a leaf node is created
        """
        for i in range(num_trees):
            tree = self._create_tree(training_set_size, min_nodes)
            self.tree_list.append(tree)

    def _create_tree(self, training_set_size, min_nodes):
        """
        Creates a decision tree from a random subset of the training examples.
        :param training_set_size: Size of the training set that each decision tree is build with
        :param min_nodes: If a node contains equal or less than 'min_nodes',
                            then the recursion is terminated and a leaf node is created
        :return: Decision tree
        """
        random_subset_indices = np.random.choice(np.arange(self.X_train.shape[0]), size=training_set_size, replace=False)
        X = self.X_train[random_subset_indices]
        Y = self.Y_train[random_subset_indices]
        tree = DecisionTree(X, Y, min_nodes=min_nodes)
        return tree

# Gaussian process 

In [7]:
class GaussianProcess:

    def __init__(self, X_train, Y_train):
        """
        :param X_train: training set
        :param Y_train: outputs of the training set
        """
        self.X_train = X_train
        self.Y_train = Y_train

    def draw_from_prior(self, X_test, num_samples, length_scale):
        """
        Returns samples drawn from the prior distribution at the test inputs.
        :param X_test: test set
        :param num_samples: number of samples to be drawn
        :param length_scale: determines the smoothness of the function
        :return: samples
        """
        K__star_star = GaussianProcess.squared_exponential_kernel(X_test, X_test, length_scale)
        return np.dot(K__star_star, np.random.normal(size=(len(X_test), num_samples)))

    def draw_from_posterior(self, X_test, num_samples, length_scale):
        """
        Returns samples drawn from the posterior at the test inputs.
        :param X_test: test set
        :param num_samples: number of samples to be drawn
        :param length_scale: determines the smoothness of the function
        :return: samples
        """
        K = GaussianProcess.squared_exponential_kernel(self.X_train, self.X_train, length_scale)
        K_inverse = np.linalg.inv(K)
        K__star_star = GaussianProcess.squared_exponential_kernel(X_test, X_test, length_scale)
        K_star_ = GaussianProcess.squared_exponential_kernel(X_test, self.X_train, length_scale)
        K__star = GaussianProcess.squared_exponential_kernel(self.X_train, X_test, length_scale)
        K_conditioned = K__star_star - np.dot(np.dot(K_star_, K_inverse), K__star)
        mu = np.dot(np.dot(K_star_, K_inverse), self.Y_train)
        return mu + np.dot(K_conditioned, np.random.normal(size=(len(X_test), num_samples)))

    def predict(self, X_test, length_scale):
        """
        Predicts the output values for the provided inputs.
        :param X_test: Test set
        :param length_scale: determines the smoothness of the function
        :return: predictions, variance for the predictions
        """
        K = GaussianProcess.squared_exponential_kernel(self.X_train, self.X_train, length_scale)
        K_inverse = np.linalg.inv(K)
        K__star_star = GaussianProcess.squared_exponential_kernel(X_test, X_test, length_scale)
        K_star_ = GaussianProcess.squared_exponential_kernel(X_test, self.X_train, length_scale)
        K__star = GaussianProcess.squared_exponential_kernel(self.X_train, X_test, length_scale)
        K_conditioned = K__star_star - np.dot(np.dot(K_star_, K_inverse), K__star)
        sigma_squared = np.diag(K_conditioned)
        mu = np.dot(np.dot(K_star_, K_inverse), self.Y_train)
        return mu, sigma_squared

    def predict_cholesky(self, X_test, length_scale):
        """
        Predicts the output values for the provided inputs. Uses the Cholesky decomposition (more on that in the report)
        in order to reduce computation and improve numerical stability.
        :param X_test: Test set
        :param length_scale: determines the smoothness of the function
        :return: predictions, variance for the predictions
        """
        K = GaussianProcess.squared_exponential_kernel(self.X_train, self.X_train, length_scale)
        L = np.linalg.cholesky(K)
        K_star_ = GaussianProcess.squared_exponential_kernel(X_test, self.X_train, length_scale)
        v = np.linalg.solve(L, self.Y_train)
        w = np.linalg.solve(L.T, v)
        mu = np.dot(K_star_, w)
        q = np.linalg.solve(L, K_star_.T)
        z = np.linalg.solve(L.T, q)
        K__star_star = GaussianProcess.squared_exponential_kernel(X_test, X_test, length_scale)
        K_conditioned = K__star_star - np.dot(K_star_, z)
        sigma_squared = np.diag(K_conditioned)
        return mu, sigma_squared

    def test(self, X_test, Y_test, length_scale, use_cholesky):
        """
        Tests the algorithm on the provided test set.
        :param X_test: test set
        :param Y_test: output values of the test set
        :param length_scale: determines the smoothness of the function
        :param use_cholesky: True: use Cholesky decomposition for computation, False: do not
        :return: RMSE for the test set, predictions on the test set, variance for the predictions
        """
        if use_cholesky:
            mu, sigma = self.predict_cholesky(X_test, length_scale)
        else:
            mu, sigma = self.predict(X_test, length_scale)
        rmse = np.sqrt(np.mean(np.square(mu - Y_test)))
        return rmse, mu, sigma

    @staticmethod
    def squared_exponential_kernel(x1, x2, length_scale):
        """
        Returns the covariance matrix according to the squared exponential kernel between x1 and x2.
        :param x1: First set of input vectors
        :param x2: Second set of input vectors
        :param length_scale: Determines the smoothness of the function
        :return: Covariance matrix
        """
        dist = np.sum(x1**2, axis=1, keepdims=True) + np.sum(x2**2, axis=1) - 2 * np.dot(x1, x2.T)
        return np.exp(-(1 / (2 * length_scale**2)) * dist)

# main function

In [8]:
data = DataLoader('sarcos_inv.csv')

def sarcos_nearest_neighbors():
    X_train, Y_train, X_cv, Y_cv, X_test, Y_test = data.get_data_split([0.6, 0.2, 0.2], normalize=True)
    print('Nearest neighbors, sarcos:')
    print('\tThis will take about 30 seconds, please wait...')
    model = NearestNeighbors(X_train, Y_train)
    #k = model.get_best_k(X_cv, Y_cv, 10)
    rmse, _ = model.test(X_test, Y_test, 4)
    print('\tRMSE: ' + str(rmse))


def sarcos_linear_regression():
    X_train, Y_train, X_cv, Y_cv, X_test, Y_test = data.get_data_split([0.6, 0.2, 0.2], normalize=True)
    print('Linear regression, sarcos:')
    print('\tThis will take about 5 seconds, please wait...')
    model = LinearRegression(X_train, Y_train)
    model.train(10000, 0.1, print_iterations=False)
    rmse, prediction = model.test(X_test, Y_test)
    rmse_train, _ = model.test(X_train, Y_train)
    print('\tRMSE: ' + str(rmse))


def sarcos_regression_forest():
    X_train, Y_train, X_cv, Y_cv, X_test, Y_test = data.get_data_split([0.6, 0.2, 0.2], normalize=False)
    print('Regression forests, sarcos:')
    print('\tThis will take about 6 minutes, please wait...')
    model = RandomForest(X_train, Y_train, num_trees=30, training_set_size=4000, min_nodes=5)
    mean, std = model.predict_all(X_test)
    rmse = np.sqrt(np.mean(np.square(mean - Y_test)))
    print('\tRMSE: ' + str(rmse))


def sarcos_gaussian_processes():
    X_train, Y_train, X_cv, Y_cv, X_test, Y_test = data.get_data_split([0.6, 0.2, 0.2], normalize=True)
    print('Gaussian processes, sarcos:')
    print('\tThis will take about 15 minutes and use a lot of memory, please wait...')
    model = GaussianProcess(X_train, Y_train)
    rmse, mean, var = model.test(X_test, Y_test, length_scale=0.9, use_cholesky=True)
    print('\tRMSE: ' + str(rmse))


sarcos_nearest_neighbors()
sarcos_linear_regression()
sarcos_regression_forest()
sarcos_gaussian_processes()

Nearest neighbors, sarcos:
	This will take about 30 seconds, please wait...
	RMSE: 3.3386142376528443
Linear regression, sarcos:
	This will take about 5 seconds, please wait...
Stopping early
	RMSE: 5.460017555852863
Regression forests, sarcos:
	This will take about 6 minutes, please wait...


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


	RMSE: 4.7163346809985445
Gaussian processes, sarcos:
	This will take about 15 minutes and use a lot of memory, please wait...
	RMSE: 5.438570800663122
